In [20]:
import pandas as pd
ticks = pd.read_csv ('ticks.csv', sep="|")
ticks['gvkey'] = ticks['gvkey'].fillna(0.0).astype(int)
ticks['Permco'] = ticks['Permco'].fillna(0.0).astype(int)
ticks

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey
0,ABN AMRO INCORPORATED,NaN,ABN,9/29/1998,9/15/2006,31989,15504
1,"AUBREY G. LANSTON & CO., INC.",Citigroup Inc.,C (NYSE),5/19/1960,4/17/2000,20483,3243
2,BA Securities,Bank of America Corporation,BAC (NYSE),4/18/1994,9/30/1997,437,7647
3,Banc One,JPMorgan Chase & Co. (Acquired by JPMorgan Chase),JPM (NYSE),4/1/1999,8/1/2004,20436,2968
4,Bank of America,Bank of America Corporation,BAC (NYSE),11/17/1971,4/15/1994,437,7647
...,...,...,...,...,...,...,...
107,"WERTHEIM SCHRODER & CO., INC.",Schroder Wertheim & Co. Inc.,NaN,6/24/1988,11/8/1990,0,0
108,WESTPAC POLLOCK GOV'T SECURITIES INC,Westpac Banking Corporation,WBK (NYSE),2/4/1987,6/27/1990,22027,15362
109,"WHITE, WELD & CO INC.","Merrill Lynch & Co., Inc.",NaN,2/26/1976,4/18/1978,21190,7267
110,"YAMAICHI INT'L (AMERICA), INC.",Yamaichi Securities Company Limited,NaN,9/29/1988,12/4/1997,0,0


In [12]:
import pandas as pd
import wrds
import config
from datetime import datetime

db = wrds.Connection(wrds_username=config.WRDS_USERNAME)

Loading library list...
Done


In [56]:
def fetch_financial_data(pgvkey, start_date, end_date):
    """
    Fetch financial data for a given ticker and date range from the CCM database in WRDS.
    
    :param gvkey: The gvkey symbol for the company.
    :param start_date: The start date for the data in YYYY-MM-DD format.
    :param end_date: The end date for the data in YYYY-MM-DD format or 'Current'.
    :return: A DataFrame containing the financial data.
    """
    if not gvkey:  # Skip if no ticker is available
        return pd.DataFrame()
    
    # If the end date is 'Current', replace it with today's date
    if end_date == 'Current':
        end_date = datetime.today().strftime('%Y-%m-%d')
    
    # Convert dates to the correct format if necessary
    start_date = pd.to_datetime(start_date).strftime('%Y-%m-%d')
    end_date = pd.to_datetime(end_date).strftime('%Y-%m-%d')
    
        
    query = f"""
    SELECT datadate, at AS total_assets, lt AS book_debt, ceq AS book_equity, csho*prcc_f AS market_equity, gvkey, conm
    FROM comp.funda as cst
    WHERE cst.gvkey = '{str(pgvkey).zfill(6)}'
    AND cst.datadate BETWEEN '{start_date}' AND '{end_date}'
    AND indfmt='INDL'
    AND datafmt='STD'
    AND popsrc='D'
    AND consol='C'
    """
    data = db.raw_sql(query)
    return data

Get all data for primary dealers

In [57]:
empty_tickers = []
prim_dealers = pd.DataFrame()
# Iterate over DataFrame rows and fetch data for each ticker
for index, row in ticks.iterrows():
    gvkey = row['gvkey']
    start_date = row['Start Date']
    end_date = row['End Date']      # Formatting date for the query
    
    # Fetch financial data for the ticker if available
    new_data = fetch_financial_data(gvkey, start_date, end_date)
    if isinstance(new_data, tuple):
        empty_tickers.append({row['Ticker']:gvkey})
    else:
        prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
print(empty_tickers)

C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\3471547033.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\3471547033.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\3471547033.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is

[]


Get data from all firms in Compustat

In [59]:
prim_dealers = prim_dealers.drop_duplicates()
prim_dealers

,datadate,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,1993-12-31,50782.297,47213.898,3226.500,7121.765625,008007,WELLS FARGO & CO
1,1994-12-31,59315.898,55469.500,3334.400,7226.264375,008007,WELLS FARGO & CO
2,1995-12-31,72134.375,66822.250,5009.797,11641.080000,008007,WELLS FARGO & CO
3,1996-12-31,80175.375,74111.188,5875.398,16038.580500,008007,WELLS FARGO & CO
4,1997-12-31,88540.188,81518.000,6834.199,29396.486250,008007,WELLS FARGO & CO
...,...,...,...,...,...,...,...
3,2001-12-31,531713.000,517166.000,9767.000,24997.940000,015504,ABN-AMRO HOLDINGS NV
4,2002-12-31,582984.873,567686.209,10450.400,25655.008000,015504,ABN-AMRO HOLDINGS NV
5,2003-12-31,705982.489,684869.917,15411.170,38457.492840,015504,ABN-AMRO HOLDINGS NV
6,2004-12-31,823953.817,797891.813,19230.729,44400.773200,015504,ABN-AMRO HOLDINGS NV


In [61]:
yearly_sum_pd = prim_dealers.groupby('datadate').agg({
    'total_assets': 'sum',
    'book_debt': 'sum',
    'book_equity': 'sum',
    'market_equity': 'sum'
}).reset_index()
yearly_sum_pd

,datadate,total_assets,book_debt,book_equity,market_equity
0,1960-12-31,8.835297e+03,4.505152e+03,746.347,0.000000e+00
1,1961-12-31,9.856233e+03,5.006815e+03,771.122,8.476998e+02
2,1962-12-31,1.039177e+04,5.191507e+03,811.380,1.936547e+03
3,1963-12-31,1.135703e+04,9.798979e+03,1377.017,1.962843e+03
4,1964-12-31,1.885544e+04,1.670331e+04,1818.247,2.841011e+03
...,...,...,...,...,...
153,2022-12-31,1.966493e+07,1.821493e+07,1321360.741,1.257698e+06
154,2023-03-31,2.271969e+06,2.178447e+06,23685.000,4.714652e+04
155,2023-10-31,6.666081e+06,6.280510e+06,361929.000,4.375408e+05
156,2023-11-30,5.790516e+04,4.810262e+04,9709.785,7.464621e+03


In [5]:
def pull_CRSP_Comp_Link_Table(wrds_username=config.WRDS_USERNAME):
    sql_query = """
        SELECT 
            gvkey, lpermco AS permco, linktype, linkprim, linkdt, linkenddt, tic
        FROM 
            crsp.ccmxpf_linkhist
        WHERE 
            substr(linktype,1,1)='L' AND 
            (linkprim ='C' OR linkprim='P')
        """
    db = wrds.Connection(wrds_username=wrds_username)
    ccm = db.raw_sql(sql_query, date_cols=["linkdt", "linkenddt"])
    db.close()
    return ccm

In [ ]:
pull_CRSP_Comp_Link_Table()

In [35]:
linktable = pd.read_csv('linktable.csv')
linktable

,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,1000,A & E PLASTIK PAK INC,AE.2,3089.0,P,01,LU,25881,23369,11/13/1970,6/30/1978
1,1001,A & M FOOD SERVICES INC,AMFD.,5812.0,P,01,LU,10015,6398,9/20/1983,7/31/1986
2,1002,AAI CORP,AAIC.1,3825.0,C,01,LC,10023,22159,12/14/1972,6/5/1973
3,1003,A.A. IMPORTING CO INC,ANTQ,5712.0,C,01,LU,10031,6672,12/7/1983,8/16/1989
4,1004,AAR CORP,AIR,5080.0,P,01,LU,54594,20000,4/24/1972,E
...,...,...,...,...,...,...,...,...,...,...,...
31944,349994,CLEARMIND MEDICINE INC,CMND,2834.0,P,01,LC,23514,59438,11/15/2022,E
31945,350681,GETNET ADQUIRENCIA E,GET,7374.0,P,90,LC,22205,58855,10/22/2021,3/31/2023
31946,351038,QUOIN PHARMACEUTICALS LTD,QNRX,2834.0,P,90,LC,16161,55612,10/29/2021,E
31947,352262,COOL COMPANY LTD,CLCO,4400.0,P,01,LC,23773,59507,3/17/2023,E


In [36]:
merged = pd.merge(ticks, linktable, left_on='gvkey', right_on='gvkey')
merged

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,ABN AMRO INCORPORATED,NaN,ABN,9/29/1998,9/15/2006,31989,15504,ABN-AMRO HOLDINGS NV,ABNYY,6020.0,P,90,LC,84766,31989,5/21/1997,4/24/2008
1,"AUBREY G. LANSTON & CO., INC.",Citigroup Inc.,C (NYSE),5/19/1960,4/17/2000,20483,3243,CITIGROUP INC,C,6199.0,P,01,LC,70519,20483,10/29/1986,E
2,Citigroup,Citigroup Inc.,C (NYSE),6/15/1961,Current,20483,3243,CITIGROUP INC,C,6199.0,P,01,LC,70519,20483,10/29/1986,E
3,BA Securities,Bank of America Corporation,BAC (NYSE),4/18/1994,9/30/1997,437,7647,BANK OF AMERICA CORP,BAC,6020.0,P,01,LC,59408,3151,12/14/1972,E
4,Bank of America,Bank of America Corporation,BAC (NYSE),11/17/1971,4/15/1994,437,7647,BANK OF AMERICA CORP,BAC,6020.0,P,01,LC,59408,3151,12/14/1972,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"SMITH BARNEY, HARRIS UPHAM & CO.,INC",Primerica Corporation,PRI,8/22/1979,8/31/1998,22177,1414,PRIMERICA INC,PRI,6311.0,C,00X,LC,10241,22177,1/1/1950,1/30/1962
94,SECURITY PACIFIC NATIONAL BANK,Security Pacific Corporation,6020,12/11/1986,1/17/1991,4205,9577,SECURITY PACIFIC CORP,7448B,6020.0,P,01,LU,60839,4205,12/14/1972,4/30/1992
95,SOUTHERN CALIF S&L ASSOC,California Federal Savings and Loan Association,NaN,6/7/1983,8/5/1983,5884,2612,CALIFORNIA FED BANCORP INC,CAL.2,6035.0,P,01,LC,66456,5884,3/29/1983,1/3/1997
96,TD SECURITIES (USA) LLC,The Toronto-Dominion Bank,"TD (NYSE, TSX)",2/11/2014,Current,29152,15706,TORONTO DOMINION BANK,TD,6020.0,P,01,LC,83835,29152,8/30/1996,E


### Get all data from broker dealers with SIC code 6211 or 6221 that are not in list of primary dealers


In [103]:
linked_bd = linktable[((linktable['sic'] == 6211) | (linktable['sic'] == 6221)) & (~linktable['gvkey'].isin(ticks['gvkey'].tolist()))]
linked_bd

,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
167,1148,ADVEST GROUP INC,ADV.2,6211.0,P,01,LU,64514,420,11/21/1980,2/28/2001
987,1967,BACHE GROUP INC,BAC.,6211.0,P,01,LU,54893,25401,5/31/1972,6/11/1981
1379,2330,BOSTON CO INC -CL B,BSTNB,6211.0,C,00X,LU,18877,22881,1/1/1975,1/30/1975
1380,2330,BOSTON CO INC -CL B,BSTNB,6211.0,C,01,LU,18877,22881,1/31/1975,9/30/1981
1475,2432,BROWN (ROBERT C.) & CO INC,RCBI,6211.0,P,01,LU,19538,6441,9/30/1983,11/30/1989
...,...,...,...,...,...,...,...,...,...,...,...
30429,178073,B. RILEY FINANCIAL INC,RILY,6211.0,P,03,LC,15540,55369,7/16/2015,E
30479,178519,LPL FINANCIAL HOLDINGS INC,LPLA,6211.0,P,01,LC,12400,53576,11/18/2010,E
30548,178848,DIRECT MARKETS HOLDINGS CORP,MKTSQ,6211.0,P,01,LC,92312,52781,10/16/2007,9/13/2012
31100,185518,GLOBAL BROKERAGE INC,GLBR,6211.0,C,01,LC,12446,53607,12/2/2010,E


In [105]:
empty_tickers = []
broker_dealers = pd.DataFrame()
for index, row in linked_bd.iterrows():
    gvkey = row['gvkey']
    start_date = '1960-01-01'
    end_date = '2012-12-31'    
    new_data = fetch_financial_data(gvkey, start_date, end_date)
    if isinstance(new_data, tuple):
        empty_tickers.append({row['conm']:gvkey})
    else:
        broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
print(empty_tickers)

C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\1247283076.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\1247283076.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\1247283076.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA en

[]


In [106]:
broker_dealers = broker_dealers.drop_duplicates()
broker_dealers

,datadate,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,1999-12-31,842.252,729.293,NaN,NaN,209386,VAN DER MOOLEN NV
1,2000-12-31,876.311,726.460,NaN,NaN,209386,VAN DER MOOLEN NV
2,2001-12-31,3631.252,3064.614,538.155,1098.808600,209386,VAN DER MOOLEN NV
3,2002-12-31,1567.193,1174.844,327.342,825.624310,209386,VAN DER MOOLEN NV
4,2003-12-31,1114.079,818.679,253.830,333.861110,209386,VAN DER MOOLEN NV
...,...,...,...,...,...,...,...
15,1996-09-30,965.177,875.935,89.242,81.929250,001148,ADVEST GROUP INC
16,1997-09-30,1078.839,973.186,105.653,227.151465,001148,ADVEST GROUP INC
17,1998-09-30,1233.271,1109.604,123.667,182.335875,001148,ADVEST GROUP INC
18,1999-09-30,1462.621,1327.485,135.136,162.881250,001148,ADVEST GROUP INC


In [107]:
yearly_sum_bd = broker_dealers.groupby('datadate').agg({
    'total_assets': 'sum',
    'book_debt': 'sum',
    'book_equity': 'sum',
    'market_equity': 'sum'
}).reset_index()
yearly_sum_bd = yearly_sum_bd.rename(columns={'total_assets':'total_assets_bd', 'book_debt':'book_debt_bd', 'book_equity':'book_equity_bd','market_equity':'market_equity_bd' })
yearly_sum_bd

,datadate,total_assets_bd,book_debt_bd,book_equity_bd,market_equity_bd
0,1960-12-31,64.700,30.700,0.000,0.000000
1,1961-12-31,75.200,39.984,0.000,0.000000
2,1962-12-31,596.288,536.342,59.895,0.000000
3,1963-12-31,624.431,561.682,23.477,0.000000
4,1964-12-31,716.440,649.862,23.733,0.000000
...,...,...,...,...,...
366,2012-03-31,15.491,8.235,7.256,26.673150
367,2012-06-30,3561.298,3203.698,357.600,175.774794
368,2012-09-30,40689.854,32587.463,7684.875,13429.889470
369,2012-11-30,36293.541,32162.737,3436.015,3448.171520


In [108]:
merged_pd_bd = pd.merge(yearly_sum_pd, yearly_sum_bd, left_on='datadate', right_on='datadate')
merged_pd_bd.index = pd.to_datetime(merged_pd_bd['datadate'])
merged_pd_bd['year'] = pd.to_datetime(merged_pd_bd['datadate']).dt.year
merged_pd_bd = merged_pd_bd.groupby('year').agg({
    'total_assets': 'sum',
    'book_debt': 'sum',
    'book_equity': 'sum',
    'market_equity': 'sum',
    'total_assets_bd': 'sum',
    'book_debt_bd': 'sum',
    'book_equity_bd': 'sum',
    'market_equity_bd': 'sum'
})
merged_pd_bd

,total_assets,book_debt,book_equity,market_equity,total_assets_bd,book_debt_bd,book_equity_bd,market_equity_bd
year,,,,,,,,
1960,8.835297e+03,4.505152e+03,746.347,0.000000e+00,64.700,30.700,0.000,0.000000
1961,9.856233e+03,5.006815e+03,771.122,8.476998e+02,75.200,39.984,0.000,0.000000
1962,1.039177e+04,5.191507e+03,811.380,1.936547e+03,596.288,536.342,59.895,0.000000
1963,1.135703e+04,9.798979e+03,1377.017,1.962843e+03,624.431,561.682,23.477,0.000000
1964,1.885544e+04,1.670331e+04,1818.247,2.841011e+03,716.440,649.862,23.733,0.000000
1965,2.057146e+04,1.826910e+04,1920.637,2.882466e+03,692.547,620.037,24.344,82.826538
1966,2.191825e+04,1.952541e+04,2033.901,2.583758e+03,877.911,796.532,80.678,119.232469
1967,2.458797e+04,2.208529e+04,2122.413,2.702707e+03,1331.803,1138.242,190.997,596.893982
1968,2.771266e+04,2.507688e+04,2225.453,3.620185e+03,1145.109,925.955,214.180,904.362257


In [110]:
merged_pd_bd['tot_assets_rat'] = merged_pd_bd['total_assets']/(merged_pd_bd['total_assets_bd']+merged_pd_bd['total_assets'])
merged_pd_bd['book_debt_rat'] = merged_pd_bd['book_debt']/(merged_pd_bd['book_debt_bd']+merged_pd_bd['book_debt'])
merged_pd_bd['book_equity_rat'] = merged_pd_bd['book_equity']/(merged_pd_bd['book_equity_bd']+merged_pd_bd['book_equity'])
merged_pd_bd['market_equity_rat'] = merged_pd_bd['market_equity']/(merged_pd_bd['market_equity_bd']+merged_pd_bd['market_equity'])
merged_pd_bd[['tot_assets_rat', 'book_debt_rat', 'book_equity_rat','market_equity_rat']].mean()

tot_assets_rat       0.909538
book_debt_rat        0.908265
book_equity_rat      0.919713
market_equity_rat    0.912978
dtype: float64

Get data from banks
6011 - Federal Reserve Banks
6021 - National Commercial Banks
6022 - State Commercial Banks
6029 - Commercial Banks, nec (not elsewhere classified)
6081 - Foreign Bank and Branches and Agencies
6082 - Foreign Trade and International Banking Institutions

In [153]:
bank_sic = [6011,6021,6022,6029,6081,6082,6020] # these are not all the codes, in the process of looking these up
linked_banks = linktable[(linktable['sic'].isin(bank_sic)) & (~linktable['gvkey'].isin(ticks['gvkey'].tolist()))]
linked_banks.drop_duplicates()

,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
197,1178,AFFILIATED BANKSHARES COLO,AFBK.,6020.0,C,00X,LU,11173,94,1/1/1974,1/30/1974
198,1178,AFFILIATED BANKSHARES COLO,AFBK.,6020.0,P,01,LU,11173,94,1/31/1974,11/30/1992
309,1290,ALLIED BANCSHARES INC-TX,ALBN,6020.0,P,01,LC,11870,151,1/31/1974,1/29/1988
310,1290,ALLIED BANCSHARES INC-TX,ALBN,6020.0,C,00X,LC,11870,151,1/1/1974,1/30/1974
485,1458,AMERICAN FLETCHER CORP,AFLT,6020.0,C,01,LU,12969,106,12/14/1972,1/23/1987
...,...,...,...,...,...,...,...,...,...,...,...
31673,260758,ATLANTIC SOUTHERN FINL GRP,ASFN,6020.0,C,01,LC,91955,52130,4/16/2007,1/28/2011
31703,264418,1ST CENTURY BANCSHARES INC,FCTY,6020.0,P,01,LC,92870,53112,12/18/2008,6/30/2016
31721,266160,CITIZENS COMMUNITY BANCORP,CZWI,6020.0,P,01,LC,91609,51445,11/1/2006,E
31807,285313,BANCO SANTANDER BRASIL -ADR,BSBR,6020.0,P,90,LC,93054,53230,10/7/2009,E


In [144]:
def get_comparison_group_data(linktable_df, start_date, end_date):
    empty_tickers = []
    comparison_group_df = pd.DataFrame()
    for index, row in linktable_df.iterrows():
        gvkey = row['gvkey']   
        new_data = fetch_financial_data(gvkey, start_date, end_date)
        if isinstance(new_data, tuple):
            empty_tickers.append({row['conm']:gvkey})
        else:
            comparison_group_df = pd.concat([new_data, comparison_group_df], axis=0)
    print('Empty Tickers: %s' % empty_tickers)
    return comparison_group_df

In [145]:
banks = get_comparison_group_data(linked_banks, '1960-01-01', '2012-12-31')
banks = banks.drop_duplicates()

C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\2771333044.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  comparison_group_df = pd.concat([new_data, comparison_group_df], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\2771333044.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  comparison_group_df = pd.concat([new_data, comparison_group_df], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\2771333044.py:10: FutureWarning: The behavior of DataFrame concatenation wit

Empty Tickers: []


In [146]:
banks

,datadate,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,2007-12-31,60887.708,55149.327,5738.349,NaN,285313,BANCO SANTANDER BRASIL -ADR
1,2008-12-31,126915.378,105415.604,21497.497,NaN,285313,BANCO SANTANDER BRASIL -ADR
2,2009-12-31,181332.899,141581.540,39750.591,52929.469060,285313,BANCO SANTANDER BRASIL -ADR
3,2010-12-31,225279.708,181167.271,44107.581,51638.506400,285313,BANCO SANTANDER BRASIL -ADR
4,2011-12-31,214680.884,172789.011,41881.694,30876.314260,285313,BANCO SANTANDER BRASIL -ADR
...,...,...,...,...,...,...,...
13,1987-12-31,2536.776,2354.405,171.473,121.224000,001178,AFFILIATED BANKSHARES COLO
14,1988-12-31,2616.952,2439.173,167.005,97.241375,001178,AFFILIATED BANKSHARES COLO
15,1989-12-31,2611.094,2425.620,174.703,122.208000,001178,AFFILIATED BANKSHARES COLO
16,1990-12-31,2738.097,2545.413,181.912,109.897250,001178,AFFILIATED BANKSHARES COLO


In [152]:
linked_banks['sic'].value_counts()

sic
6020.0    1413
6022.0       1
Name: count, dtype: int64

In [147]:
yearly_sum_banks = banks.groupby('datadate').agg({
    'total_assets': 'sum',
    'book_debt': 'sum',
    'book_equity': 'sum',
    'market_equity': 'sum'
}).reset_index()
yearly_sum_banks = yearly_sum_banks.rename(columns={'total_assets':'total_assets_bd', 'book_debt':'book_debt_bd', 'book_equity':'book_equity_bd','market_equity':'market_equity_bd' })
yearly_sum_banks

,datadate,total_assets_bd,book_debt_bd,book_equity_bd,market_equity_bd
0,1960-12-31,78599.073,41552.276,6361.886,0.000000
1,1961-12-31,86176.330,43121.544,6858.341,0.000000
2,1962-12-31,92296.289,48642.078,7224.936,10988.304739
3,1963-12-31,126363.247,115456.199,9614.007,12355.733646
4,1964-12-31,140929.862,129064.851,10389.785,13035.880255
...,...,...,...,...,...
165,2012-03-31,4534458.104,4320424.337,180131.325,162161.210470
166,2012-06-30,4502.981,4026.934,476.043,221.196470
167,2012-09-30,1489418.503,1396915.511,86393.809,131908.920423
168,2012-10-31,1534.168,1441.064,93.104,0.000000


In [148]:
merged_pd_banks = pd.merge(yearly_sum_pd, yearly_sum_banks, left_on='datadate', right_on='datadate')
merged_pd_banks.index = pd.to_datetime(merged_pd_banks['datadate'])
merged_pd_banks['year'] = pd.to_datetime(merged_pd_banks['datadate']).dt.year
merged_pd_banks = merged_pd_banks.groupby('year').agg({
    'total_assets': 'sum',
    'book_debt': 'sum',
    'book_equity': 'sum',
    'market_equity': 'sum',
    'total_assets_bd': 'sum',
    'book_debt_bd': 'sum',
    'book_equity_bd': 'sum',
    'market_equity_bd': 'sum'
})
merged_pd_banks

,total_assets,book_debt,book_equity,market_equity,total_assets_bd,book_debt_bd,book_equity_bd,market_equity_bd
year,,,,,,,,
1960,8.835297e+03,4.505152e+03,746.347,0.000000e+00,7.859907e+04,4.155228e+04,6361.886,0.000000e+00
1961,9.856233e+03,5.006815e+03,771.122,8.476998e+02,8.617633e+04,4.312154e+04,6858.341,0.000000e+00
1962,1.039177e+04,5.191507e+03,811.380,1.936547e+03,9.229629e+04,4.864208e+04,7224.936,1.098830e+04
1963,1.135703e+04,9.798979e+03,1377.017,1.962843e+03,1.263632e+05,1.154562e+05,9614.007,1.235573e+04
1964,1.885544e+04,1.670331e+04,1818.247,2.841011e+03,1.409299e+05,1.290649e+05,10389.785,1.303588e+04
1965,2.057146e+04,1.826910e+04,1920.637,2.882466e+03,1.574110e+05,1.445361e+05,11053.722,1.208050e+04
1966,2.191825e+04,1.952541e+04,2033.901,2.583758e+03,1.718237e+05,1.576631e+05,11627.191,1.364269e+04
1967,2.458797e+04,2.208529e+04,2122.413,2.702707e+03,1.987475e+05,1.832923e+05,12516.733,1.494430e+04
1968,2.771266e+04,2.507688e+04,2225.453,3.620185e+03,2.244083e+05,2.077676e+05,13352.231,2.166649e+04


In [151]:
merged_pd_banks['tot_assets_rat'] = merged_pd_banks['total_assets']/(merged_pd_banks['total_assets_bd']+merged_pd_banks['total_assets'])
merged_pd_banks['book_debt_rat'] = merged_pd_banks['book_debt']/(merged_pd_banks['book_debt_bd']+merged_pd_banks['book_debt'])
merged_pd_banks['book_equity_rat'] = merged_pd_banks['book_equity']/(merged_pd_banks['book_equity_bd']+merged_pd_banks['book_equity'])
merged_pd_banks['market_equity_rat'] = merged_pd_banks['market_equity']/(merged_pd_banks['market_equity_bd']+merged_pd_banks['market_equity'])
merged_pd_banks[['tot_assets_rat', 'book_debt_rat', 'book_equity_rat','market_equity_rat']]
# these ratios are between 0.1 and 0.2 off of the paper's numbers for banks

,tot_assets_rat,book_debt_rat,book_equity_rat,market_equity_rat
year,,,,
1960,0.101051,0.097816,0.104998,NaN
1961,0.102634,0.104030,0.101072,1.000000
1962,0.101197,0.096436,0.100964,0.149831
1963,0.082464,0.078232,0.125286,0.137084
1964,0.118005,0.114588,0.148939,0.178940
1965,0.115581,0.112214,0.148033,0.192640
1966,0.113131,0.110196,0.148883,0.159231
1967,0.110094,0.107535,0.144982,0.153154
1968,0.109918,0.107698,0.142862,0.143166


In [179]:
tick_inv = ticks.copy()
tick_inv['End Date'] = pd.to_datetime(tick_inv['End Date'].replace({'Current': datetime.today()}))

# Now apply the filter
tick_inv_filtered = tick_inv[tick_inv['End Date'] <= '2012-12-31']
tick_inv_filtered = tick_inv_filtered[tick_inv_filtered['gvkey'] == 0]
tick_inv_filtered

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey
12,Becker Paribas Inc.,Banque Paribas,BNP (Euronext Paris),11/17/1971,1984-09-10,0,0
19,"Brophy, Gestal, Knight",NaN,NaN,5/8/1987,1988-06-19,0,0
23,CARROLL MCENTEE & MCGINLEY INC.,NaN,NaN,9/29/1976,1994-05-06,0,0
24,"C.F. CHILDS & CO., INC",NaN,NaN,5/19/1960,1965-06-29,0,0
26,CHEMICAL SECURITIES INC,Chemical Bank Corporation,CHL (NYSE),5/19/1960,1996-03-31,0,0
42,Drexel Burnham Lambert,NaN,NaN,5/19/1960,1990-03-28,0,0
43,"D.W. RICH & CO., INC",NaN,NaN,5/19/1960,1969-12-31,0,0
44,EASTBRIDGE CAPITAL INC.,NaN,NaN,6/18/1992,1998-05-29,0,0
45,FI Dupont,NaN,NaN,12/12/1968,1973-07-18,0,0
50,FIRST PENNCO SEC. INC.,NaN,NaN,3/7/1974,1980-08-27,0,0


In [177]:
tick_inv_filtered[['Primary Dealer', 'Start']].tolist()

['Becker Paribas Inc.',
 'Brophy, Gestal, Knight',
 'CARROLL MCENTEE & MCGINLEY INC.',
 'C.F. CHILDS & CO., INC',
 'CHEMICAL SECURITIES INC',
 'Drexel Burnham Lambert',
 'D.W. RICH & CO., INC',
 'FI Dupont',
 'FIRST PENNCO SEC. INC.',
 'HARRIS GOVERNMENT SECURITIES',
 'IRVING SECURITIES, INC.',
 "KLEINWORT BENSON  GOV'T SEC., INC.",
 'MALON S. ANDRUS INC.',
 "NUVEEN GOV'T SEC. INC.",
 'NY Hanseatic',
 'SECOND DISTRICT SECURITIES CO., INC',
 'Securities Groups',
 'S.G. WARBURG & CO., INC.',
 'THOMSON MCKINNON SECURITIES INC.',
 'WERTHEIM SCHRODER & CO., INC.']

Creation of table 2 writeup: Table 2 was the first table our group was tasked with replicating. This table involves getting numbers such as total assets or market equity from CRSP Compustat database for different groups of firms, with the limitation that the firms be US domestic and have data available in CRSP Compustat. These groups were comprised of the primary dealers of the New York Fed, which are detailed in table A.1, all broker dealers, all banks, and all firms in Compustat. There were three time ranges that we observed, 1960-2012, 1960-1990, and 1990-2012. Replicating table 2 turned out to be a lot more involved than initially expected, due largely to the condition that we use the data of holding companies of the primary dealers rather than of the actual primary dealer. This required us to manually go through each individual dealer in the list of 112, and look up information about their holding company during a specified time period where they were considered a primary dealer. This was not a simple task because there were many cases where holding companies shifted multiple time within the specified period - forcing us to try and guess how the original author of the paper made this decision. We ended up choosing the holding company that held the primary dealer for the longest period of time during the specified interval. An alternative method would be to have multiple holding company data pulls for a single primary dealer, which may be the most accurate and most time consuming methodology. What was also a challenge was that firms seemed to have multiple names that they went by, making research and lookups of these firms difficult and easy to make an error.

After finishing the main lookup file, ticks.csv, which including all primary dealers listed in table A.1, their holding companies, their start and end dates, and their permcos and gvkeys (identifiers for CRSP and Compustat respectively) we then focused on pulling data from the WRDS database. After establishing a connection with WRDS we were able to query data from multiple sources. We made use of the CRSP Compustat historical link table to do most of this research, as well as some holding companies lookups on Google and chatGPT. We queried a full list of link table data from the WRDS website and downloaded it in Excel format to be able to filter and make notes. We also queried the link table from the WRDS historical table and used this table to help merge SIC codes with our completed ticks.csv dataset. For the financials dataset, we opted to query data by gvkey since the data for table 2 was specifically for company level financials which are found through Compustat.
  
After having the primary dealer data in the proper format of total assets, total book debt, book equity, and market equity, it was now time to create our subgroups and query data for them. The first one we worked on was the broker dealer dataset, which, as specified in the paper, included all primary dealers in our main dataset plus any firms with the SIC code 6211 or 6221. Using the historical linktable and our primary dealer linktable we created our subset and then used it to query financial data from Compustat. Now with our two different subsets of data, we merged them on the date key and then performed our ratio calculation. This calculation was the total value of primary dealers divided by the sum of the total value of primary dealers and the total value of non-primary dealer broker dealers. We then repeated this methodology for the banks comparison group which was generally comprised of firms with the SIC code 6020. We found that the bank ratios were farther from the paper than the broker dealer ratios. This could be for a multitude of reasons, potentially from differences in how holding companies were handled. 